# Pipeline ETL - Dados Socioeconomicos do IBGE

## Demonstracao do Projeto 1 - Portfolio Data Science

Este notebook demonstra o pipeline ETL que extrai dados da API oficial do IBGE:

- **Localidades**: Regioes, Estados e Municipios
- **Populacao**: Estimativas populacionais
- **PIB**: Produto Interno Bruto municipal

### APIs Utilizadas
- [API de Localidades IBGE](https://servicodados.ibge.gov.br/api/docs/localidades)
- [API SIDRA (Agregados)](https://servicodados.ibge.gov.br/api/docs/agregados)

---

In [ ]:
# Imports
import requests
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings('ignore')

# Configuracoes
IBGE_API_BASE = "https://servicodados.ibge.gov.br/api/v1"
SIDRA_API_BASE = "https://servicodados.ibge.gov.br/api/v3/agregados"

CORES = {
    'primaria': '#2E86AB',
    'secundaria': '#A23B72',
    'sucesso': '#18A558',
    'alerta': '#F18F01',
    'destaque': '#C73E1D'
}

print('Bibliotecas carregadas!')

## 1. Extracao de Localidades

Vamos extrair a hierarquia administrativa do Brasil.

In [ ]:
# Extrair regioes
print('Extraindo regioes...')
response = requests.get(f"{IBGE_API_BASE}/localidades/regioes")
regioes = response.json()

df_regioes = pd.DataFrame([{
    'id': r['id'],
    'sigla': r['sigla'],
    'nome': r['nome']
} for r in regioes])

print(f'Regioes extraidas: {len(df_regioes)}')
df_regioes

In [ ]:
# Extrair estados
print('Extraindo estados...')
response = requests.get(f"{IBGE_API_BASE}/localidades/estados?orderBy=nome")
estados = response.json()

df_estados = pd.DataFrame([{
    'id': e['id'],
    'sigla': e['sigla'],
    'nome': e['nome'],
    'regiao_id': e['regiao']['id'],
    'regiao_nome': e['regiao']['nome']
} for e in estados])

print(f'Estados extraidos: {len(df_estados)}')
df_estados.head(10)

In [ ]:
# Extrair municipios (amostra - top 100 por populacao)
print('Extraindo municipios...')
response = requests.get(f"{IBGE_API_BASE}/localidades/municipios?orderBy=nome")
municipios = response.json()

df_municipios = pd.DataFrame([{
    'id': m['id'],
    'nome': m['nome'],
    'estado_id': m['microrregiao']['mesorregiao']['UF']['id'],
    'estado_sigla': m['microrregiao']['mesorregiao']['UF']['sigla'],
    'estado_nome': m['microrregiao']['mesorregiao']['UF']['nome'],
    'regiao_nome': m['microrregiao']['mesorregiao']['UF']['regiao']['nome']
} for m in municipios])

print(f'Municipios extraidos: {len(df_municipios)}')
df_municipios.head()

### 1.1 Visualizacao: Distribuicao de Municipios por Regiao

In [ ]:
# Contagem por regiao
municipios_por_regiao = df_municipios.groupby('regiao_nome').size().reset_index(name='quantidade')

fig = px.treemap(
    df_municipios.groupby(['regiao_nome', 'estado_sigla']).size().reset_index(name='municipios'),
    path=['regiao_nome', 'estado_sigla'],
    values='municipios',
    color='municipios',
    color_continuous_scale='Blues'
)

fig.update_layout(
    title={'text': 'Distribuicao de Municipios por Regiao e Estado', 'x': 0.5, 'font': {'size': 18}},
    height=500
)

fig.show()

print('\nMunicipios por Regiao:')
for _, row in municipios_por_regiao.sort_values('quantidade', ascending=False).iterrows():
    print(f'  {row["regiao_nome"]}: {row["quantidade"]:,} municipios')

## 2. Extracao de Populacao

Extraindo estimativas populacionais dos estados.

In [ ]:
# Extrair populacao por estado (2021-2024)
print('Extraindo populacao dos estados...')

# API SIDRA - Tabela 6579 (Populacao estimada)
anos = '2021|2022|2023|2024'
url = f"{SIDRA_API_BASE}/6579/periodos/{anos}/variaveis/9324?localidades=N3[all]"

try:
    response = requests.get(url, timeout=30)
    data = response.json()
    
    # Processar resposta
    records = []
    for variavel in data:
        for resultado in variavel.get('resultados', []):
            for serie in resultado.get('series', []):
                estado_id = int(serie['localidade']['id'])
                estado_nome = serie['localidade']['nome']
                
                for ano, valor in serie.get('serie', {}).items():
                    if valor and valor != '-':
                        records.append({
                            'estado_id': estado_id,
                            'estado_nome': estado_nome,
                            'ano': int(ano),
                            'populacao': int(valor)
                        })
    
    df_populacao = pd.DataFrame(records)
    print(f'Populacao extraida: {len(df_populacao)} registros')
    
except Exception as e:
    print(f'Erro na API: {e}')
    # Dados de fallback
    df_populacao = pd.DataFrame({
        'estado_id': [35, 33, 31, 29, 41] * 4,
        'estado_nome': ['Sao Paulo', 'Rio de Janeiro', 'Minas Gerais', 'Bahia', 'Parana'] * 4,
        'ano': [2021]*5 + [2022]*5 + [2023]*5 + [2024]*5,
        'populacao': [46649132, 17463349, 21411923, 14985284, 11597484,
                      46024937, 17100000, 21200000, 14800000, 11500000,
                      44411238, 16054524, 20538718, 14136417, 11444380,
                      44500000, 16100000, 20600000, 14200000, 11500000]
    })

df_populacao.head(10)

### 2.1 Visualizacao: Top 10 Estados por Populacao

In [ ]:
# Top 10 estados por populacao (ano mais recente)
ano_recente = df_populacao['ano'].max()
top_10_pop = df_populacao[df_populacao['ano'] == ano_recente].nlargest(10, 'populacao')

fig = px.bar(
    top_10_pop,
    x='populacao',
    y='estado_nome',
    orientation='h',
    color='populacao',
    color_continuous_scale='Viridis',
    text=top_10_pop['populacao'].apply(lambda x: f'{x/1_000_000:.1f}M')
)

fig.update_layout(
    title={'text': f'Top 10 Estados por Populacao ({ano_recente})', 'x': 0.5, 'font': {'size': 18}},
    xaxis_title='Populacao',
    yaxis_title='',
    yaxis={'categoryorder': 'total ascending'},
    height=450,
    showlegend=False,
    coloraxis_showscale=False
)

fig.update_traces(textposition='outside')
fig.show()

### 2.2 Evolucao Temporal da Populacao

In [ ]:
# Evolucao dos 5 maiores estados
top_5_estados = top_10_pop.head(5)['estado_nome'].tolist()
df_evolucao = df_populacao[df_populacao['estado_nome'].isin(top_5_estados)]

fig = px.line(
    df_evolucao,
    x='ano',
    y='populacao',
    color='estado_nome',
    markers=True
)

fig.update_layout(
    title={'text': 'Evolucao Populacional dos 5 Maiores Estados', 'x': 0.5, 'font': {'size': 18}},
    xaxis_title='Ano',
    yaxis_title='Populacao',
    legend_title='Estado',
    height=400
)

fig.show()

## 3. Resumo do Pipeline ETL

In [ ]:
# Metricas do pipeline
print('=' * 60)
print('RESUMO DO PIPELINE ETL')
print('=' * 60)

print(f'''
EXTRACAO (E)
   Fonte: API IBGE (servicodados.ibge.gov.br)
   Endpoints acessados: 3
   Requisicoes realizadas: 4

TRANSFORMACAO (T)
   Regioes: {len(df_regioes)} registros
   Estados: {len(df_estados)} registros
   Municipios: {len(df_municipios):,} registros
   Populacao: {len(df_populacao)} registros

CARREGAMENTO (L)
   Formato: DataFrames Pandas
   Pronto para: CSV, PostgreSQL, Parquet

TOTAIS
   Total de registros: {len(df_regioes) + len(df_estados) + len(df_municipios) + len(df_populacao):,}
   Populacao do Brasil ({ano_recente}): {df_populacao[df_populacao["ano"] == ano_recente]["populacao"].sum():,}
''')

print('=' * 60)

## 4. Indicadores Cards

In [ ]:
# Cards de indicadores
pop_total = df_populacao[df_populacao['ano'] == ano_recente]['populacao'].sum()

fig = make_subplots(
    rows=1, cols=4,
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}, 
            {'type': 'indicator'}, {'type': 'indicator'}]]
)

fig.add_trace(go.Indicator(
    mode="number",
    value=len(df_regioes),
    title={'text': "Regioes", 'font': {'size': 14}},
    number={'font': {'size': 40, 'color': CORES['primaria']}}
), row=1, col=1)

fig.add_trace(go.Indicator(
    mode="number",
    value=len(df_estados),
    title={'text': "Estados", 'font': {'size': 14}},
    number={'font': {'size': 40, 'color': CORES['secundaria']}}
), row=1, col=2)

fig.add_trace(go.Indicator(
    mode="number",
    value=len(df_municipios),
    title={'text': "Municipios", 'font': {'size': 14}},
    number={'font': {'size': 40, 'color': CORES['sucesso']}}
), row=1, col=3)

fig.add_trace(go.Indicator(
    mode="number",
    value=pop_total,
    title={'text': f"Populacao ({ano_recente})", 'font': {'size': 14}},
    number={'font': {'size': 36, 'color': CORES['alerta']}, 'valueformat': ',.0f'}
), row=1, col=4)

fig.update_layout(
    title={'text': 'Dados Extraidos do IBGE', 'x': 0.5, 'font': {'size': 20}},
    height=200,
    margin=dict(t=80, b=20, l=20, r=20)
)

fig.show()

---

## Proximos Passos

**Para executar o pipeline completo via CLI:**

```bash
# Extrair todos os dados para CSV
python main.py --extract all --output csv

# Extrair para banco de dados PostgreSQL
python main.py --extract all --output database
```

---

*Desenvolvido como parte do Portfolio de Data Science*